# 06 Scrapy爬虫框架

内容导航：
1. Scrapy架构
2. 安装
3. 创建爬虫项目
4. 创建爬虫模块
5. 选择器
6. 命令行工具
7. 定义Item
8. 翻页功能
9. Item Pipeline
10. 内置数据存储
11. 内置图片和文件下载Pipeline
12. 启动爬虫
13. 强化爬虫

## 6.1 Scrapy架构

Scrapy框架的架构如图所示：

![scrapy架构](images/scrapy-arch.jpeg)

Scrapy中的各大组件及其功能为：

* **Scrapy引擎（Engine）**：引擎负责控制数据流在系统的各组件中流动，并在响应动作发生时触发事件。
* **调度器（Scheduler）**：调度器从引擎接收Request并将其入队，以便之后引擎请求request时提供给引擎。
* **下载器（Downloader）**：负责获取页面数据并提供给引擎，后者再提供给Spider
* 采集爬虫（Spider）：*用户编写*用于分析Response并提取Item或额外跟进的URL的类。每个Spider负责处理一个（些）特定的网站。
* **数据项管线（Item Pipeline）**：负责处理被Spider提取出来的Item，如清理、验证和持久化（到文件或数据库）。
* 下载器中间件（Downloader Middlewares）：引擎和下载器之间的特定钩子（hook）或者拦截器，处理Downloader传递给引擎的Response。这提供了一种通过插入自定义代码来扩展Scrapy功能的机制。
* Spider中间件（Spider Middlewares）：引擎和Spider之间的特定钩子或拦截器，处理Spider的输入（response,也就是来自下载器的输出）和输出（Items及Request）。这也提供了一种通过插入自定义代码来扩展Scrapy功能的机制。

## 6.2 安装

### 1. Windows

1. 安装pywin32
2. 安装pyOpenSSL
3. 安装lxml
4. 安装Scrapy: `pip install scrapy`

### 2. Unix/Linux

```
pip install scrapy
```

In [10]:
!pip show scrapy

Name: Scrapy
Version: 1.7.3
Summary: A high-level Web Crawling and Web Scraping framework
Home-page: https://scrapy.org
Author: Scrapy developers
Author-email: None
License: BSD
Location: /anaconda3/lib/python3.7/site-packages
Requires: service-identity, queuelib, six, Twisted, cssselect, pyOpenSSL, PyDispatcher, lxml, w3lib, parsel
Required-by: 


## 6.3 创建爬虫项目

以爬取[cnblogs](https://www.cnblogs.com)项目为例,爬取文章的链接、时间、标题和摘要。

In [5]:
!scrapy startproject CNblogSpider

New Scrapy project 'CNblogSpider', using template directory '/anaconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/xiaobai/Workspace/labs/python-samples/web_spider/CNblogSpider

You can start your first spider with:
    cd CNblogSpider
    scrapy genspider example example.com


In [14]:
!tree CNblogSpider/ 

CNblogSpider/
├── CNblogSpider
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-37.pyc
│   │   ├── items.cpython-37.pyc
│   │   ├── pipelines.cpython-37.pyc
│   │   └── settings.cpython-37.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-37.pyc
│       │   └── cnblogs_spider.cpython-37.pyc
│       └── cnblogs_spider.py
├── ch06_scrapy_test.ipynb
├── papers.json
└── scrapy.cfg

4 directories, 16 files


## 6.4 创建爬虫模块
   
   首先编写爬虫模块，爬虫模块的代码都放置于spiders文件中。爬虫模块是用于从单个网址或者多个网址爬取数据的类，其应该包含初始页面的URL，以及跟进网页链接、分析页面内容和提取数据。
   创建一个Spider类，需要继承scrapy.Spider类，并定义以下三个类属性：
   
   1. name: 为Spider指定唯一的名称
   2. start_urls: Spider启动是进行爬取的入口URL列表
   3. parse(): 负责解析下载的响应数据（response)、提取数据（生成Item）以及生成需要进一步处理的URL的Request对象
   
   在项目根目录的CNblogSpider/spiders目录下创建cnblogs_spider.py文件。代码如下：

In [5]:
!cat CNblogSpider/CNblogSpider/spiders/cnblogs_spider.py

from scrapy import Spider
from scrapy.selector import Selector
from CNblogSpider.items import CnblogspiderItem

class CnblogsSpider(Spider):
    # 爬虫的名称
    name = "cnblogs"
    allowed_domains = ['cnblogs.com']
    start_urls = ['https://www.cnblogs.com/qiyeboy/']
    
    def parse(self, response):
        papers = response.xpath('.//div[@class="day"]')
        for paper in papers:
            # 提取链接
            link = paper.xpath('.//*[@class="postTitle"]/a/@href')
            link = link.extract()[0].strip()
            # 提取标题
            title = paper.xpath('.//*[@class="postTitle"]/a/text()')
            title = title.extract()[0].strip()
            # 提取时间
            time = paper.xpath('.//*[@class="dayTitle"]/a/text()')
            time = time.extract()[0].strip()
            # 提取摘要
            summary = paper.xpath('.//*[@class="c_b_p_desc"]/text()')
            summary = summary.extract()[0].strip()
            item = CnblogspiderItem(link=link, title=title, 
               

## 6.5 选择器

Scrapy有自己的一套数据提取机制，称为选择器（selector），可使用XPath、CSS和正则表达式选择HTML文件中的某个部分。

### Selector的用法

Selector对象（scrapy.selector.Selector）有四个基本方法：

1. Xpath(query)：传入XPath表达式query，返回该表达式所应所以节点的Selector列表
2. css(query)：传入CSS选择器表达式query, 返回同上
3. extract()：序列化选中的节点为Unicode字符串并返回列表
4. re(regex)：根据传入的正则表达式对数据进行提取，返回Unicode字符串列表。regex可以是一个已编译的正则表达式模式对象（pattern），也可以为正则表达式字符串。

可以使用scrapy shell检查XPath、CSS选择器和正则表达式是否正确，便于实现parse方法。

### HTML解析实现

通过分析网页结构，XPath表达式如下：

* 所有文章： `.//*[@class='day']`
* 文章发表时间：`.//*[@class='dayTitle']/a/text()`
* 文章标题内容：`.//*[@class='postTitle']/a/text()`
* 文章摘要内容：`.//*[@class='postCon']/div/text()`
* 文章链接：`.//*[@class='postTitle']/a/@href`

## 6.6 命令行工具

### 全局命令

全局命令如下：
* startproject，创建项目, 完整命令为：`scrapy startproject Cnblogs`
* settings，在项目根目录运行时，输出项目设置，否则输出Scrapy默认设置，如：
  ```
  scrapy settings --get BOT_NAME
  ```
* runspider，在未创建项目的情况下，运行一个编好的spider模块。如：
  ```
  scrapy runspider cnblogs_spider.py
  ```
* shell，用于启动Scrapy shell交互模式，以帮助编写HTML解析代码。如：
  ```
  scrapy shell 'http://www.cnblogs.com/qieyeboy/
  ```
* fetch，使用Scrapy下载器下载给定的URL，并将获取到的内容以标准输出。如：
  ```
  scrapy fetch --nolog 'http://www.cnblogs.com/qiyeboy'
  scrapy fetch --nolog --headers 'http://www.cnblogs.com/qiyeboy'
  ```
* bench，用于运行benchmark测试
* verson，查看Scrapy版本信息。

In [6]:
!scrapy version -v

Scrapy       : 1.7.3
lxml         : 4.3.2.0
libxml2      : 2.9.9
cssselect    : 1.0.3
parsel       : 1.5.1
w3lib        : 1.20.0
Twisted      : 19.7.0
Python       : 3.7.3 (default, Mar 27 2019, 16:54:48) - [Clang 4.0.1 (tags/RELEASE_401/final)]
pyOpenSSL    : 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019)
cryptography : 2.6.1
Platform     : Darwin-17.7.0-x86_64-i386-64bit


### 项目命令

项目命令如下：

* crawl，运行指定的爬虫， 完整命令：`scrapy crawl cnblogs`
* check，运行contract检查，完整命令：`scrapy check -l`
* list，列出当前项目中所有可用的spider，每行输出一个spider：`scrapy list`
* edit，使用设定的编辑器编辑给定的spider，完整命令：`scrapy edit cnblogs`
* parse，获取给定的URL并使用相应的spider分析处理。语法：
  ```
  scrapy parse <url> [options]
  ```
  支持的选项：
  * --spider=SPIDER: 强制使用特定的spider
  * --a NAME=VALUE: 设置spider参数
  * --callback or -c: spider中用于解析response的回调函数
  * --pipelines: 在pipeline中处理item
  * --rules or -r: 使用CrawlSpider规则发现用于解析response的回调函数
  * --noitems: 不显示爬取到的item
  * --nocolour: 避免着色
  * --depth or -d: 指定根据链接请求的层数
  * --verbose or -v: 显示每个请求的详细信息
* genspider，在当前项目中创建spider,可根据事先定义好的模版生成spider，语法：
  ```
  scrapy [-t template] <name> <domain>
  ```
* deploy，将项目部署到Scrapyd服务，语法：
  ```
  scrapy deploy [<target:project> | -l <target> | -L]
  ```


## 6.7 定义Item

爬取的主要目标就是从非结构行的数据源提取结构性数据。scrapy提供Item来作为容器，用来保存爬取到的数据，提供了类似自定的API以及用于声明可用字段的简单语法。

在Scrapy项目的items.py文件中定义保存目标数据的Item类，该类需要基础scrapy.Item。代码如下：

In [8]:
!cat CNblogSpider/CNblogSpider/items.py

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy

class CnblogspiderItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    link = scrapy.Field()
    time = scrapy.Field()
    title = scrapy.Field()
    summary = scrapy.Field()


## 6.8 翻页功能

翻页功能的实现，本质上是构造Request并提交给Scrapy引擎的过程。形如：

```
yield scrapy.Request(url=next_page_url, callback=self.parse)
```
其中，Request对象构造方法中url参数为请求链接，callback为回调方法。回调方法用来指定由谁解析此Request请求的响应。

## 6.9 构建Item Pipeline

当Item在Spider中被收集后，它将会被传递到Item Pipeline，一些组件会按照一定的顺序执行对Item的处理。

Item Pipeline主要有以下典型的应用：
* 清理HTML数据
* 验证爬取数据的合法性，检查Item是否包含某些字段
* 查重并丢弃
* 将爬取结果保存到文件或者数据库中

### 定制Item Pipeline

定制Item Pipeline非常简单，每个Item Pipeline组件是一个独立的python类，必须实现process_item方法，方法原型如下：
```
process_item(self, item, spider)
```
每个Item Pipeline组件都需要重写（实现）该方法，该方法必须返回一个Item（或派生类）对象，或者抛出DropItem异常，被丢弃的Item不会被之后的Pipeline组件所处理。

参数说明：
* Item对象是被爬取的Item
* Spider对象代表着爬取该Item的Spider

自定义Item Pipeline代码位于项目目录的pipelines.py文件中。

### 激活 Item Pipeline

定制完Item Pipeline,它是无法工作的，需要进行激活。要启用一个Item Pipeline组件，必须将其添加到settings.py中的ITEM_PIPELINES变量中。代码如下：
```
ITEM_PIPELINES = {
    'CNblogSpider.pipelines.CNblogSpiderPipeline': 300, 
}
```

ITEM_PIPELINES变量中可以配置多个Item Pipeline组件，分配给每个类的整数确定了其运行顺序，item按数字从低到高的顺序通过Item pipeline,通常将这些数定义在0~1000范围内。

## 6.10 内置数据存储

除了使用Item Pipeline实现存储功能，Scrapy内置了一些简单的存储方式，生成一个带有爬取数据的输出文件，称为feed，支持多种序列化方式。其支持的类型有：

* JSON，FEED_FORMAT:json，内置输出类：JsonItemExporter
* JSON lines, FEED_FORMAT:jsonlines，内置输出类：JsonLinesItemExporter
* CSV，FEED_FORMAT:csv, 内置输出类：CsvItemExporter
* XML, FEED_FORMAT:xml, 内置输出类：XmlItemExporter
* Pickle, FEED_FORMAT:pickle, 内置输出类：PickleItemExporter
* Marshal, FEED_FORMAT:marshal, 内置输出类：shalItemExporter

使用方式：如果想将爬取结果保存为CSV格式，在项目根目录下输入命令:
```
scrapy crawl cnblogs -o papers.csv
```

## 6.11 内置图片和文件下载

有时爬取文本数据的同时也要下载相应的图片和文件，Scrapy为下载Item中包含的文件提供了一种可重用的Item Pipeline。这些Pipeline称为MediaPipeline，更为具体的Pipeline有FilesPipeline和ImagesPipeline。这两种Pipeline都实现了以下特性：

* 避免重新下载已经下载过的数据
* 可指定存储的位置和方式

此外，ImagesPipeline还提供了额外特征：
* 将下载的图片转为通用格式（如JPG）和色彩模式（RGB）
* 生成缩略图
* 检测图像的宽/高，确保其满足需求